<a href="https://colab.research.google.com/github/ProjectXMG999/SQL_project/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install stripe

from flask import Flask, render_template, request, redirect, url_for, session
import sqlite3
import stripe

app = Flask(__name__)
app.secret_key = 'secret_key'

# Connect to the database
conn = sqlite3.connect('ecommerce.db')
c = conn.cursor()

# Configure Stripe
stripe.api_key = 'stripe_secret_key'

# Define the routes
@app.route('/')
def index():
    # Display the product catalog page
    categories = c.execute('SELECT * FROM categories').fetchall()
    products = c.execute('SELECT * FROM products').fetchall()
    return render_template('catalog.html', categories=categories, products=products)

@app.route('/register', methods=['GET', 'POST'])
def register():
    # Register a new customer
    if request.method == 'POST':
        name = request.form['name']
        email = request.form['email']
        password = request.form['password']
        c.execute('INSERT INTO customers (name, email, password) VALUES (?, ?, ?)', (name, email, password))
        conn.commit()
        return redirect(url_for('login'))
    else:
        return render_template('register.html')

@app.route('/login', methods=['GET', 'POST'])
def login():
    # Log in an existing customer
    if request.method == 'POST':
        email = request.form['email']
        password = request.form['password']
        customer = c.execute('SELECT * FROM customers WHERE email=? AND password=?', (email, password)).fetchone()
        if customer:
            session['customer_id'] = customer[0]
            return redirect(url_for('cart'))
        else:
            return render_template('login.html', error='Invalid email or password')
    else:
        return render_template('login.html')

@app.route('/logout')
def logout():
    # Log out the current customer
    session.pop('customer_id', None)
    return redirect(url_for('index'))

@app.route('/cart', methods=['GET', 'POST'])
def cart():
    # Display the shopping cart page
    if 'customer_id' not in session:
        return redirect(url_for('login'))
    if request.method == 'POST':
        product_id = request.form['product_id']
        quantity = request.form['quantity']
        c.execute('INSERT INTO cart_items (customer_id, product_id, quantity) VALUES (?, ?, ?)', (session['customer_id'], product_id, quantity))
        conn.commit()
        return redirect(url_for('cart'))
    else:
        cart_items = c.execute('SELECT * FROM cart_items WHERE customer_id=?', (session['customer_id'],)).fetchall()
        products = []
        total_price = 0
        for cart_item in cart_items:
            product = c.execute('SELECT * FROM products WHERE product_id=?', (cart_item[2],)).fetchone()
            products.append(product)
            total_price += product[3] * cart_item[3]
        return render_template('cart.html', products=products, cart_items=cart_items, total_price=total_price)

@app.route('/checkout', methods=['GET', 'POST'])
def checkout():
    # Process the order and payment
    if 'customer_id' not in session:
        return redirect(url_for('login'))
    if request.method == 'POST':
        # Create a new order
        c.execute('INSERT INTO orders (customer_id, total_price) VALUES (?, ?)', (session['customer_id'], request.form['total_price']))
        order_id = c.lastrowid
        # Add the order items
        cart_items = c.execute('SELECT * FROM cart_items WHERE customer_id=?', (session['customer_id'],)).fetchall()
        for cart_item in cart_items:
            c.execute('INSERT INTO order_items (order_id, product_id, quantity) VALUES (?, ?, ?)', (order_id, cart_item[2], cart_item[3]))
        # Process the payment
        token = request.form['stripeToken']
        charge = stripe.Charge.create(
            amount=int(request.form['total_price'] * 100),
            currency='usd',
            description='E-commerce Order',
            source=token,
        )
        # Clear the cart
        c.execute('DELETE FROM cart_items WHERE customer_id=?', (session['customer_id'],))
        conn.commit()
        return redirect(url_for('order_confirmation', order_id=order_id))
    else:
        cart_items = c.execute('SELECT * FROM cart_items WHERE customer_id=?', (session['customer_id'],)).fetchall()
        products = []
        total_price = 0
        for cart_item in cart_items:
            product = c.execute('SELECT * FROM products WHERE product_id=?', (cart_item[2],)).fetchone()
            products.append(product)
            total_price += product[3] * cart_item[3]
        return render_template('checkout.html', products=products, cart_items=cart_items, total_price=total_price)

@app.route('/order_confirmation/<int:order_id>')
def order_confirmation(order_id):
    # Display the order confirmation page
    order = c.execute('SELECT * FROM orders WHERE order_id=?', (order_id,)).fetchone()
    order_items = c.execute('SELECT * FROM order_items WHERE order_id=?', (order_id,)).fetchall()
    products = []
    for order_item in order_items:
        product = c.execute('SELECT * FROM products WHERE product_id=?', (order_item[2],)).fetchone()
        products.append(product)
    return render_template('order_confirmation.html', order=order, order_items=order_items, products=products)

# Run the app
if __name__ == '__main__':
    app.run(debug=True)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.4/255.4 kB 5.9 MB/s eta 0:00:00
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
